In [ ]:
# %%
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import sys
import time

def log_error_to_file(filename, error, error_log_file='errors.txt'):
    """Appends the name of a failed notebook and the error to a log file."""
    with open(error_log_file, 'a', encoding='utf-8') as f:
        f.write(f"Notebook: {filename}\n")
        f.write(f"Error: {str(error)}\n")
        f.write("-" * 20 + "\n")

def run_specific_notebooks(notebook_files, notebook_dir='.'):
    """
    Executes a specific list of Jupyter Notebooks.

    If a notebook fails, its name is logged to errors.txt and the script continues.
    For each successfully executed notebook, it creates a new notebook with the
    suffix '_executed' which contains the code with all the cell outputs.

    Args:
        notebook_files (list): A list of .ipynb filenames to execute.
        notebook_dir (str): The path to the directory where the notebooks are located.
                            Defaults to the current directory.
    """
    if not os.path.isdir(notebook_dir):
        print(f"Error: Directory not found at '{notebook_dir}'")
        return

    print(f"Executing notebooks from directory: {os.path.abspath(notebook_dir)}\n")

    # --- Setup the execution preprocessor ---
    # This preprocessor will execute the notebook's code.
    # You can specify a timeout in seconds for each cell.
    # If a cell takes longer than the timeout, it will raise an exception.
    executor = ExecutePreprocessor(timeout=600, kernel_name='python3')

    total_notebooks = len(notebook_files)
    executed_count = 0
    error_count = 0

    for i, filename in enumerate(notebook_files):
        # Check if the file exists before attempting to run
        input_path = os.path.join(notebook_dir, filename)
        if not os.path.exists(input_path):
            print(f"[{i+1}/{total_notebooks}] --- Skipping: {filename} (File not found) ---")
            print("-" * (len(filename) + 20) + "\n")
            error_count += 1
            log_error_to_file(filename, "File not found.")
            continue
            
        output_filename = filename.replace('.ipynb', '_executed.ipynb')
        output_path = os.path.join(notebook_dir, output_filename)

        print(f"[{i+1}/{total_notebooks}] --- Executing: {filename} ---")
        start_time = time.time()

        try:
            # --- Read the notebook file ---
            with open(input_path, 'r', encoding='utf-8') as f:
                nb = nbformat.read(f, as_version=4)

            # --- Execute the notebook ---
            # The second argument is a dictionary for resources,
            # 'metadata': {'path': '...'} sets the working directory for the notebook.
            executor.preprocess(nb, {'metadata': {'path': notebook_dir}})

            # --- Write the executed notebook to a new file ---
            with open(output_path, 'w', encoding='utf-8') as f:
                nbformat.write(nb, f)

            end_time = time.time()
            execution_time = end_time - start_time
            print(f"  \033[92m✔ Success!\033[0m Output saved to: {output_filename} (Took {execution_time:.2f}s)")
            executed_count += 1

        except CellExecutionError as e:
            # --- Handle errors during cell execution ---
            end_time = time.time()
            execution_time = end_time - start_time
            error_message = f"Error executing notebook: {filename} (Failed after {execution_time:.2f}s)"
            print(f"  \033[91m✖ {error_message}\033[0m")
            print("  Logging error to errors.txt and continuing...")
            log_error_to_file(filename, e)
            error_count += 1

        except Exception as e:
            # --- Handle other potential errors (e.g., file reading) ---
            print(f"  \033[91m✖ An unexpected error occurred\033[0m with {filename}: {e}")
            print("  Logging error to errors.txt and continuing...")
            log_error_to_file(filename, e)
            error_count += 1
        
        print("-" * (len(filename) + 20) + "\n")
    
    print("--- Execution Summary ---")
    print(f"Total notebooks processed: {total_notebooks}")
    print(f"\033[92mSuccessfully executed: {executed_count}\033[0m")
    print(f"\033[91mFailed or Skipped: {error_count}\033[0m")
    if error_count > 0:
        print("Details for failed/skipped notebooks have been logged to errors.txt")


if __name__ == '__main__':
    # --- Script Entry Point ---
    
    # --- Define the list of notebooks to run ---
    # Add the filenames of your Jupyter Notebooks to this list.
    ipynb_files_to_run = [
        "EfficientNetB0_224_32.ipynb",
        "EfficientNetB0_224_64.ipynb",
        "EfficientNetB0_224_128_512_256_128.ipynb",
        "EfficientNetB4_224_32_64_02.ipynb",
        "EfficientNetB4_224_32.ipynb",
        "EfficientNetB4_224_64.ipynb",
        "EfficientNetB4_224_128_512_256_128.ipynb",
        "EfficientNetB7_224_32_512_256_128.ipynb",
        "EfficientNetB7_224_64_512_256_128.ipynb",
    ]

    # You can specify the directory where the notebooks are located.
    # Default is the current directory ('.').
    notebook_directory = '.'

    if not ipynb_files_to_run:
        print("The list of notebooks to run is empty. Please add filenames to the 'ipynb_files_to_run' list.")
    else:
        run_specific_notebooks(ipynb_files_to_run, notebook_directory)



# %%